In [1]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import squareform, pdist
from scipy.stats import zscore
from sklearn import linear_model
from math import cos, sin, pi, isnan
import xlsxwriter
import nibabel as nib
from nilearn.glm.second_level import SecondLevelModel
from nilearn.image import load_img, threshold_img
from nilearn.glm import threshold_stats_img
from nilearn import plotting
import matplotlib.pyplot as plt
from nltools.data import Brain_Data

/home/haiyanwu/miniconda3/lib/python3.9/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "
/home/haiyanwu/miniconda3/lib/python3.9/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  warn('The nilearn.glm module is experimental. '


In [2]:
topdir='/media/haiyanwu/HDD4/fmri2019/'
evdir=topdir+'2.GLM/event_files/'
prepdir=topdir+'fmriprep-deriv/fmriprep/'
glmdir=topdir+'2.GLM/3.GLM_SPM/results/'
maskdir=topdir+'3.ROIextraction/masks/'

In [3]:
subList=[102,103,105,107,108,110,111,112,113,114,115,116,117,118,119,120,121,122,123,125,127,128,129,130,131,132]
len(subList)

26

# compute modelRDMs
### models: gridness; only consistency; only money; diagonal; par

In [10]:

all=pd.read_csv(topdir+'4.DDM-related/fMRI_sessionData_DDM.csv',sep=',')

metric_model='seuclidean'
for subno in subList:
    dist=[]

    data=all.loc[all['subj']==subno,:]
    data=data[data['RT']<=4]
    data=data.groupby(['former_diff','diff','islie']).size().reset_index()
    data=data[data['former_diff']!=0]
    data.reset_index(inplace=True)
    #data['diff']=zscore(data['diff'])
    #data['former_diff']=zscore(data['former_diff'])

    # 1. gridness
    dist1=pdist(data.loc[:,['islie','diff','former_diff']],metric=metric_model)

    # 2. orth1
    data['orth1_con']=data['former_diff']
    data['orth1_mon']=data['diff']
    data.loc[data['islie']==0,'orth1_con']=0
    data.loc[data['islie']==1,'orth1_mon']=0
    dist2=pdist(data.loc[:,['islie','orth1_con','orth1_mon']],metric=metric_model)

    # 3. orth2
    data['orth2_con']=data['former_diff']
    data['orth2_mon']=data['diff']
    data.loc[data['islie']==0,'orth2_mon']=0
    data.loc[data['islie']==1,'orth2_con']=0
    dist3=pdist(data.loc[:,['islie','orth2_con','orth2_mon']],metric=metric_model)

    # 4. par1
    data['par1']=data['former_diff']*(-1)
    data.loc[data['islie']==1,'par1']=data.loc[data['islie']==1]['diff']
    dist4=pdist(data.loc[:,['islie','par1']],metric=metric_model)
    

    # 5. par2
    data['par2']=data['former_diff']
    data.loc[data['islie']==1,'par2']=data.loc[data['islie']==1]['diff']
    dist5=pdist(data.loc[:,['islie','par2']],metric=metric_model)

    # 6. par3
    data['par3']=data['former_diff']
    data.loc[data['islie']==0,'par3']=data.loc[data['islie']==0]['diff']
    dist6=pdist(data.loc[:,['islie','par3']],metric=metric_model)
    

    # 7. par4
    data['par4']=data['former_diff']
    data.loc[data['islie']==0,'par4']=data.loc[data['islie']==0]['diff']*(-1)
    dist7=pdist(data.loc[:,['islie','par4']],metric=metric_model)

    # 8. rotgrid1
    data['rotgrid1_con']=data['former_diff']
    data['rotgrid1_mon']=data['diff']
    data.loc[data['islie']==0,'rotgrid1_con']=data.loc[data['islie']==0]['diff']*(-1)
    data.loc[data['islie']==0,'rotgrid1_mon']=data.loc[data['islie']==0]['former_diff']
    dist8=pdist(data.loc[:,['islie','rotgrid1_con','rotgrid1_mon']],metric=metric_model)

    # 9. rotgrid2
    data['rotgrid2_con']=data['former_diff']
    data['rotgrid2_mon']=data['diff']
    data.loc[data['islie']==0,'rotgrid2_con']=data.loc[data['islie']==0]['diff']
    data.loc[data['islie']==0,'rotgrid2_mon']=data.loc[data['islie']==0]['former_diff']*(-1)
    dist9=pdist(data.loc[:,['islie','rotgrid2_con','rotgrid2_mon']],metric=metric_model)

    

    # 10. only consistency
    dist10=pdist(data.loc[:,['islie','former_diff']],metric=metric_model)

    # 11. only money
    dist11=pdist(data.loc[:,['islie','diff']],metric=metric_model)

    # 12. diagonal
    data['diag']=data['diff']/2+data['former_diff']/2
    

    dist12=pdist(data.loc[:,['islie','diag','diag']],metric=metric_model)




    dist.append(dist1)
    dist.append(dist2)
    dist.append(dist3)
    dist.append(dist4)
    dist.append(dist5)

    dist.append(dist6)
    dist.append(dist7)
    dist.append(dist8)
    dist.append(dist9)
    dist.append(dist10)
    dist.append(dist11)
    dist.append(dist12)
  

    subdata=pd.DataFrame(dist)
    subdata['subj']=subno
    
    subdata['model']=['grid','orth1','orth2','par1','par2','par3','par4','rotgrid1','rotgrid2','con','mon','diag']
    subdata.to_csv('12modelRDMs0925/sub-%03d.csv'%subno)

  

# RDM regression within ROIs

In [11]:
ROIs=['BA6-SMA','BA9-dlPFC','BA13-insula','BA24-dACC',
        'BA28-hip1','BA34-hip','lTPJ','BA23-PCC','OFC']
len(ROIs)

28

In [4]:
ROIs=['precuneus']

In [5]:
def drop_na(z):
    ztt=[]
    for i in z:
        if not isnan(i):
            ztt.append(i)
    return ztt

In [7]:
all=pd.read_csv(topdir+'4.DDM-related/fMRI_sessionData_DDM.csv',sep=',')
results={}
writer=pd.ExcelWriter('RDMregression-12models0925.xlsx',engine='xlsxwriter')
for subno in subList:

    modelRDM=pd.read_csv('12modelRDMs0925/sub-%03d.csv'%subno,index_col=0)
    results['sub-%03d'%subno]={}

    length=len(list(modelRDM.iloc[0,:]))-2

    grid=drop_na(list(modelRDM.iloc[0,0:length]))
    orth1=drop_na(list(modelRDM.iloc[1,0:length]))
    orth2=drop_na(list(modelRDM.iloc[2,0:length]))
    par1=drop_na(list(modelRDM.iloc[3,0:length]))
    par2=drop_na(list(modelRDM.iloc[4,0:length]))
    par3=drop_na(list(modelRDM.iloc[5,0:length]))
    par4=drop_na(list(modelRDM.iloc[6,0:length]))
    rotgrid1=drop_na(list(modelRDM.iloc[7,0:length]))
    rotgrid2=drop_na(list(modelRDM.iloc[8,0:length]))
    con=drop_na(list(modelRDM.iloc[9,0:length]))
    mon=drop_na(list(modelRDM.iloc[10,0:length]))
    diag=drop_na(list(modelRDM.iloc[11,0:length]))
            
    
    d=all[all['subj']==subno]
    d=d[d['RT']<=4]
    d=d.groupby(['former_diff','diff','islie']).size().reset_index()
    d=d[d['former_diff']!=0]
    d['combination']=list(map(lambda x,y,z:'con%d_mon%d_islie%d'%(x,y,z),d['former_diff'],d['diff'],d['islie']))



    betas=Brain_Data()
    

    for cname in d['combination'].to_list():
        fname=glmdir+'GLM6/betamap/sub-%03d/sub-%03d_%s_eff_map.nii.gz'%(subno,subno,cname)
        betas=betas.append(Brain_Data(fname))

    for roi in ROIs:
        results['sub-%03d'%subno][roi]=[]
        mask=Brain_Data(maskdir+'Mask_%s.nii'%(roi))
        brainDist = betas.apply_mask(mask).distance(metric='correlation')
    

        # perform regression
        Y=pd.DataFrame(brainDist.data)
        X=pd.DataFrame([grid,orth1,orth2,par1,par2,par3,par4,rotgrid1,rotgrid2,con,mon,diag]).T#

        lm = linear_model.LinearRegression()
        fit=lm.fit(X, Y)
        results['sub-%03d'%subno][roi]=fit.coef_[0]
    
    results['sub-%03d'%subno]=pd.DataFrame.from_dict(results['sub-%03d'%subno])
    results['sub-%03d'%subno].to_excel(writer,sheet_name='sub-%03d'%subno,startcol=0,engine='xlsxwriter')
writer.save()
    

# generate brain RDM

In [5]:
all=pd.read_csv(topdir+'4.DDM-related/fMRI_sessionData_DDM.csv',sep=',')
results={}
writer=pd.ExcelWriter('brainRDMs0925_precuneus.xlsx',engine='xlsxwriter')
for roi in ROIs:
    results[roi]={}
for subno in subList:

    
    d=all[all['subj']==subno]
    d=d[d['RT']<=4]
    d=d.groupby(['former_diff','diff','islie']).size().reset_index()
    d=d[d['former_diff']!=0]
    d['combination']=list(map(lambda x,y,z:'con%d_mon%d_islie%d'%(x,y,z),d['former_diff'],d['diff'],d['islie']))



    betas=Brain_Data()
    

    for cname in d['combination'].to_list():
        fname=glmdir+'GLM6/betamap/sub-%03d/sub-%03d_%s_eff_map.nii.gz'%(subno,subno,cname)
        betas=betas.append(Brain_Data(fname))

    for roi in ROIs:
        mask=Brain_Data(maskdir+'Mask_%s.nii'%(roi))
        brainDist = betas.apply_mask(mask).distance(metric='correlation')
        results[roi]['sub-%03d'%subno]=brainDist.data
    

for roi in ROIs:

   results[roi]=pd.DataFrame(dict([(k,pd.Series(v)) for k,v in results[roi].items()]))
   
   results[roi].to_excel(writer,sheet_name='%s'%roi,startcol=0,engine='xlsxwriter')
writer.save()

  

/home/haiyanwu/miniconda3/lib/python3.9/site-packages/nilearn/image/resampling.py:273: UserWarning: Resampling binary images with continuous or linear interpolation. This might lead to unexpected results. You might consider using nearest interpolation instead.
  warnings.warn("Resampling binary images with continuous or "
/home/haiyanwu/miniconda3/lib/python3.9/site-packages/nilearn/image/resampling.py:273: UserWarning: Resampling binary images with continuous or linear interpolation. This might lead to unexpected results. You might consider using nearest interpolation instead.
  warnings.warn("Resampling binary images with continuous or "
/home/haiyanwu/miniconda3/lib/python3.9/site-packages/nilearn/image/resampling.py:273: UserWarning: Resampling binary images with continuous or linear interpolation. This might lead to unexpected results. You might consider using nearest interpolation instead.
  warnings.warn("Resampling binary images with continuous or "
/home/haiyanwu/miniconda3/li

In [ ]:
all=pd.read_csv(topdir+'4.DDM-related/fMRI_sessionData_DDM.csv',sep=',')
results={}
writer=pd.ExcelWriter('combination.xlsx',engine='xlsxwriter')
for roi in ROIs:
    results[roi]={}
for subno in subList:

    
    d=all[all['subj']==subno]
    d=d[d['RT']<=4]
    d=d.groupby(['former_diff','diff','islie']).size().reset_index()
    d=d[d['former_diff']!=0]
    d['combination']=list(map(lambda x,y,z:'con%d_mon%d_islie%d'%(x,y,z),d['former_diff'],d['diff'],d['islie']))
    d.to_excel(writer,sheet_name='%d'%subno,startcol=0,engine='xlsxwriter')
    
writer.save()

